In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest

In [3]:
# Load your CSV
df = pd.read_csv("combined_good_coil_data_added gauge target and phases_Gauge lst 0.08.csv")

# Preview first rows
df.head()

,Coil ID,Coil Length [30ms],Master Ramp,Shape Target Second Coefficient,Stand 1 Predicted Run Force,Stand 1 Gap Stick Offset,Tension Reel Calculated Tension,Stand 1-2 Total Tension Feedback,Stand 3 - Operator Side Force,Stand 1 Run Gap Setpoint,...,Stand 4 DS Roll Force,Roll Force Hydraulics Pressure Feedback,Stand 1 Roll Force,Stand 1 Roll Force limit (g67 delayed 200 ms + g80),Stand 1 Roll Force Increase Limit (based on predicted run force),Stand 4 OS Bending Shape Trim,Stand 4 DS Bending Shape Trim,Shape Target Second Coefficient.1,X4 Gauge Target,Phase
0,5399358,148.33333,43.168007,10.0,1769.0,0.0,9.359998,44.578003,717.06900,44.30005,...,380.54993,2993.0000,1631.0,1796.0,176.0,-51.420906,24.791690,10.0,-0.098532,Phase 1
1,5399358,149.50000,43.443005,10.0,1769.0,0.0,9.373999,44.272003,730.43024,44.30005,...,375.53995,2992.5000,1628.1,1804.6,176.0,-51.345573,24.836271,10.0,-0.103068,Phase 1
2,5399358,151.33333,43.730995,10.0,1769.0,0.0,9.394003,44.633007,740.06240,44.30005,...,370.43994,2991.6667,1623.6,1808.4,176.0,-51.267693,24.873936,10.0,-0.107208,Phase 1
3,5399358,152.66667,44.005997,10.0,1769.0,0.0,9.397998,43.714000,728.77740,44.30005,...,366.50010,2991.0000,1619.8,1808.0,176.0,-51.184810,24.913872,10.0,-0.097812,Phase 1
4,5399358,154.50000,44.281000,10.0,1769.0,0.0,9.407007,43.379005,721.62366,44.30005,...,369.93005,2990.0000,1618.8,1802.6,176.0,-51.104633,24.976248,10.0,-0.091188,Phase 1


In [4]:
# Specify only the IBA signal columns (edit this list)
iba_signals = ['Shape Target Second Coefficient', 'Stand 1 Predicted Run Force', 'Stand 1 Gap Stick Offset', 'Tension Reel Calculated Tension', 
                    'Stand 1-2 Total Tension Feedback', 'Stand 3 - Operator Side Force', 'Stand 1 Run Gap Setpoint', 'Stand 1 Gap Bite Offset', 
                    'S1 Operating Bending Trim', 'Stand 2-3 Tension Reference', 'Stand 3 Predicted Run Force', 'Neet Oil Concentration', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand1', 'Stand 4 Top Current Feedback', 'Morgoil DriveTop Bearing Outflow Temp Stand3', 
                    'Stand 2-3 Total Tension Feedback', 'Stand 2 Predicted Run Force', 'Stand 4 Gap Thread Offset', 'Stand 2 - Operator Side Force', 
                    'Stand 3 Run Gap Setpoint', 'Stand 2 Total Bending Feedback', 'Stand 2 Gap Bite Offset', 'Morgoil OperBottom Bearing Outflow Temp Stand3', 
                    'Exit Tension Reel Tension Reference', 'Stand 4 Thread Gap Setpoint', 'X4 Gauge Deviation', 'Stand 4 DS Total Bending Feedback',
                    'Stand 4 Gap Stick Offset', 'Stand 4 - Operator Side Force', 'Stand 2 Gap Eccentricity Trim', 'Stand 4 Gap Operator Offset', 
                    'Stand 3 Total Bending Feedback', 'Morgoil OperTop Bearing Outflow Temp Stand1', 'Stand 1 - Operator Side Force', 'X1 Gauge Deviation', 
                    'Stand 3 Drive Speed Feedback', 'Stand 2 Gap Thread Offset', 'Stand 4 Gap Bite Offset', 'Stand 2 Drive Speed Feedback', 
                    'Stand 3 Thread Gap Setpoint', 'Stand 1 Drive Speed Feedback', 'Stand 1-3 Solution System Pressure', 'Morgoil OperBottom Bearing Outflow Temp Stand4', 
                    'Stand 2 Thread Gap Setpoint', 'Stand 2 Top Current Feedback', 'Stand 1-3 Solution Temperature', 'Stand 4 - Drive Side Force', 'Stand 1 - Drive Side Force', 
                    'AGC GE Feedforward Hardness Number', 'Stand 1 Total Bending Feedback', 'Morgoil OperBottom Bearing Outflow Temp Stand1', 'X0 Gauge Deviation', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand4', 'Stand 4 Predicted Run Force', 'Stand 3 Bottom Current Feedback', 'Stand 4 Gap Eccentricity Trim', 
                    'Morgoil DriveBottom Bearing Outflow Temp Stand1', 'Stand 2 Gap Stick Offset', 'Stand 3-4 Tension Reference', 'Stand 4 Bottom Current Feedback', 
                    'Morgoil DriveTop Bearing Outflow Temp Stand2', 'Stand 1 Bottom Current Feedback', 'S3 Operating Bending Trim', 'Morgoil DriveBottom Bearing Outflow Temp Stand4', 
                    'Stand 4 Drive Speed Feedback', 'Stand 3 Gap Stick Offset', 'Morgoil OperTop Bearing Outflow Temp Stand4', 'Stand 3 Gap Thread Offset', 
                    'Morgoil OperBottom Bearing Outflow Temp Stand2', 'Stand 3-4 Total Tension Feedback', 'Morgoil OperTop Bearing Outflow Temp Stand2', 
                    'Stand 2 Gap Operator Offset', 'Stand 2 Bottom Current Feedback', 'Stand 2 - Total Force', 'Stand 4 Solution System Pressure', 'Stand 3 Gap Eccentricity Trim', 
                    'Stand 3 Gap Bite Offset', 'Stand 2 - Drive Side Force', 'Stand 4 OS Total Bending Feedback', 'Morgoil DriveBottom Bearing Outflow Temp Stand2', 
                    'Stand 4 Run Gap Setpoint', 'Stand 1-2 Tension Reference', 'Stand 1 Gap Thread Offset', 'Stand 1 Gap Operator Offset', 'AGC Alex Dynamic Feedforward Hardness Number',
                    'Stand 3 - Drive Side Force', 'Stand 2 Run Gap Setpoint', 'Stand 1 - Total Force', 'Stand 1 Thread Gap Setpoint', 'Stand 1 Top Current Feedback', 
                    'S4 Operating Bending Trim', 'Stand 1 Gap Eccentricity Trim', 'Master Ramp.1', 'Stand 3 Top Current Feedback', 'Morgoil DriveBottom Bearing Outflow Temp Stand3', 
                    'S2 Operating Bending Trim', 'Morgoil OperTop Bearing Outflow Temp Stand3', 'Stand 3 Gap Operator Offset', 'Stand 1 Backup RPM', 'Stand 2 Backup RPM', 
                    'Stand 3 Backup RPM', 'Stand 4 Backup RPM', 'Stand 1 Top Motor RPM', 'Stand 1 Bottom Motor RPM', 'Stand 2 Top Motor RPM', 'Stand 2 Bottom Motor RPM', 
                    'Stand 3 Top Motor RPM', 'Stand 3 Bottom Motor RPM', 'Stand 4 Top Motor RPM', 'Stand 4 Bottom Motor RPM', 'Payoff Reel OS RPM', 'Payoff Reel DS RPM', 
                    'Exit Tension Reel RPM', 'Roll Force Hydraulic Tank Level Inches', 'Stand 1 OS Roll Force', 'Stand 2 OS Roll Force', 'Stand 3 OS Roll Force', 
                    'Stand 4 OS Roll Force', 'Stand 1 DS Roll Force', 'Stand 2 DS Roll Force', 'Stand 3 DS Roll Force', 'Stand 4 DS Roll Force', 'Roll Force Hydraulics Pressure Feedback',
                    'Stand 1 Roll Force', 'Stand 1 Roll Force limit (g67 delayed 200 ms + g80)', 'Stand 1 Roll Force Increase Limit (based on predicted run force)', 
                    'Stand 4 OS Bending Shape Trim', 'Stand 4 DS Bending Shape Trim', 'Shape Target Second Coefficient.1']
# print("Available columns:", df.columns.tolist())
# print("Using signals:", iba_signals)

In [5]:
# def detect_anomalies(coil_data, signal_cols):
#     """
#     Run anomaly detection for one coil:
#     - IsolationForest for overall anomaly detection
#     - IQR method for per-signal anomaly %
#     """
#     # Standardize only IBA signals
#     scaler = StandardScaler()
#     X_scaled = scaler.fit_transform(coil_data[signal_cols])
    
#     # Isolation Forest for anomaly detection
#     iso = IsolationForest(contamination=0.05, random_state=42)
#     y_pred = iso.fit_predict(X_scaled)
    
#     # Add anomaly flag (-1 = anomaly, 1 = normal)
#     coil_data["Anomaly"] = y_pred
    
#     # Overall anomaly percentage
#     anomaly_percentage = (coil_data["Anomaly"] == -1).mean() * 100
    
#     # Per-signal anomaly percentage (using IQR method)
#     anomaly_per_signal = {}
#     for sig in signal_cols:
#         q1 = coil_data[sig].quantile(0.25)
#         q3 = coil_data[sig].quantile(0.75)
#         iqr = q3 - q1 
#         lower, upper = q1 - 1.5*iqr, q3 + 1.5*iqr
#         anomaly_per_signal[sig] = ((coil_data[sig] < lower) | (coil_data[sig] > upper)).mean() * 100
    
#     return anomaly_percentage, anomaly_per_signal, coil_data

In [6]:
# results = []

# for coil_id, group in df.groupby("Coil ID"):
#     coil_pct, sig_pct, coil_with_anoms = detect_anomalies(group.copy(), iba_signals)
    
#     # Save results
#     result = {"Coil ID": coil_id, "Overall_Anomaly_%": coil_pct}
#     result.update({f"{sig}_Anomaly%": val for sig, val in sig_pct.items()})
#     results.append(result)

# results_df = pd.DataFrame(results)
# results_df

In [7]:
def detect_anomalies(coil_phase_data, signal_cols):
    """
    Run anomaly detection for one coil-phase segment:
    - IsolationForest for overall anomaly detection
    - IQR method for per-signal anomaly %
    """
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(coil_phase_data[signal_cols])
    
    iso = IsolationForest(contamination=0.05, random_state=42)
    y_pred = iso.fit_predict(X_scaled)
    
    coil_phase_data["Anomaly"] = y_pred
    anomaly_percentage = (coil_phase_data["Anomaly"] == -1).mean() * 100
    
    anomaly_per_signal = {}
    for sig in signal_cols:
        q1 = coil_phase_data[sig].quantile(0.25)
        q3 = coil_phase_data[sig].quantile(0.75)
        iqr = q3 - q1
        lower, upper = q1 - 1.5 * iqr, q3 + 1.5 * iqr
        anomaly_per_signal[sig] = ((coil_phase_data[sig] < lower) | (coil_phase_data[sig] > upper)).mean() * 100
    
    return anomaly_percentage, anomaly_per_signal, coil_phase_data


# --- Phasewise anomaly detection ---
results = []

for (coil_id, phase), group in df.groupby(["Coil ID", "Phase"]):
    coil_pct, sig_pct, coil_with_anoms = detect_anomalies(group.copy(), iba_signals)
    
    result = {
        "Coil ID": coil_id,
        "Phase": phase,
        "Overall_Anomaly_%": coil_pct
    }
    result.update({f"{sig}_Anomaly%": val for sig, val in sig_pct.items()})
    results.append(result)

results_df = pd.DataFrame(results)
results_df

KeyboardInterrupt: 

In [6]:
results_df.to_csv("coil_anomaly_isolationforest_badcoils.csv", index=False)
print("Saved to coil_anomaly_isolationforest_badcoils.csv")

Saved to coil_anomaly_isolationforest_badcoils.csv
